In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import joblib
import os

def QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end=4, layer_configs=[[128, 64, 32], [256, 128, 64, 32]], learning_rates=[0.5,0.01, 0.005, 0.001], optimizer_classes=[optim.Adam, optim.SGD]):
    def ensure_directory_exists(path):
        if not os.path.exists(path):
            os.makedirs(path)
            print(f"Created directory: {path}")
        else:
            print(f"Directory already exists: {path}")
    ensure_directory_exists(save_path)

    class NeuralNet(nn.Module):
        def __init__(self, input_size, output_size, layers):
            super(NeuralNet, self).__init__()
            self.layers = nn.ModuleList()
            self.activations = nn.ModuleDict()
            last_size = input_size
            for i, size in enumerate(layers):
                self.layers.append(nn.Linear(last_size, size))
                self.activations[f'layer{i+1}'] = nn.ReLU()
                last_size = size
            self.output_layer = nn.Linear(last_size, output_size)

        def forward(self, x):
            for i, linear in enumerate(self.layers):
                x = linear(x)
                x = self.activations[f'layer{i+1}'](x)
            return self.output_layer(x)
    data = data.sample(n=5000, random_state=42)  # 'n' is the number of items from the axis to return.


    features = data[feature_column]
    target = data[target_column]
    X = torch.tensor(features.values, dtype=torch.float32)
    y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
    X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    best_model_performance = float('-inf')
    best_models = None
    quantiles = [0.5 + (i / (number_of_quantile_one_end * 2 + 2)) for i in range(-number_of_quantile_one_end, number_of_quantile_one_end + 1)]

    for layer_sizes in layer_configs:
        for lr in learning_rates:
            for Optimizer in optimizer_classes:
                models = [NeuralNet(X_train.shape[1], 1, layer_sizes) for _ in quantiles]
                optimizers = [Optimizer(model.parameters(), lr=lr) for model in models]

                for epoch in range(200):
                    for inputs, labels in train_loader:
                        current_outputs = {}
                        for optimizer in optimizers:
                            optimizer.zero_grad()
                        for q_idx, quantile in enumerate(quantiles):
                            outputs = models[q_idx](inputs)
                            current_outputs[quantile] = outputs.detach()
                            lower_bound, upper_bound = set_bounds(quantiles, current_outputs, quantile)
                            loss = adjusted_quantile_loss(outputs, labels, quantile, lower_bound, upper_bound)
                            loss.backward(retain_graph=True if q_idx < len(quantiles) - 1 else False)
                        for optimizer in optimizers:
                            optimizer.step()

                        # Clear outputs to save memory
                        current_outputs.clear()

                #Evaluate and print R-squared after each epoch
                total_loss = 0
                total_r2 = 0
                with torch.no_grad():
                    for inputs, labels in train_loader:
                        for q_idx, model in enumerate(models):
                            model.eval()
                            outputs = model(inputs)
                            r2 = r2_score(labels.numpy(), outputs.numpy())
                            total_r2 += r2
                            loss = adjusted_quantile_loss(outputs, labels, quantiles[q_idx], lower_bound=None, upper_bound=None)  # Adjust bounds as needed
                            total_loss += loss.item()
                            print(f'Epoch {epoch}, Quantile {quantiles[q_idx]}, R2: {r2:.4f}')
                            model.train()

                # Check if current configuration is the best
                avg_loss = total_loss / len(train_loader)
                #avg_r2 = total_r2 / (len(train_loader) * len(models))

                # Check if the current configuration is the best based on loss
                if avg_loss < best_model_performance:
                    best_model_performance = avg_loss
                    best_models = [model.state_dict() for model in models]

    # Save the best models for each quantile
    if best_models:
        for idx, model_state in enumerate(best_models):
            torch.save({
                'state_dict': model_state,
                'weights': [param.detach().numpy() for param in models[idx].parameters()]
            }, os.path.join(save_path, f'model_quantile_{quantiles[idx]:.2f}.pth'))

    return quantiles, models, scaler

def adjusted_quantile_loss(outputs, targets, quantile, lower_bound=None, upper_bound=None):
    errors = targets - outputs
    basic_loss = torch.max((quantile - 1) * errors, quantile * errors)
    loss = torch.mean(basic_loss)

    if lower_bound is not None:
        crossing_penalty = torch.mean(torch.relu(lower_bound - outputs))
        loss += crossing_penalty
    if upper_bound is not None:
        crossing_penalty = torch.mean(torch.relu(outputs - upper_bound))
        loss += crossing_penalty

    return loss

def set_bounds(quantiles, current_outputs, current_quantile):
    lower_bound = None
    upper_bound = None
    idx = quantiles.index(current_quantile)
    if idx > 0:
        lower_bound = current_outputs.get(quantiles[idx - 1])
    if idx < len(quantiles) - 1:
        upper_bound = current_outputs.get(quantiles[idx + 1])
    return lower_bound, upper_bound

# Example usage remains as previously described.
def adjusted_quantile_loss(outputs, targets, quantile, lower_bound, upper_bound):
    errors = targets - outputs
    basic_loss = torch.max((quantile - 1) * errors, quantile * errors)
    loss = torch.mean(basic_loss)
    if lower_bound is not None:
        loss += torch.mean(torch.relu(lower_bound - outputs))
    if upper_bound is not None:
        loss += torch.mean(torch.relu(outputs - upper_bound))
    return loss

# Usage example
data = pd.read_csv('/home/yui/Downloads/read_and_play/p122_synthetic_1_1.1_1.2.csv')  # specify the path to your dataset
feature_column = ['battery_2#p122','gas-cc#p122','upv#p122','wind-ons#p122']
target_column = 'value'
save_path = 'model_output'
# Example usage
quantiles, models, scaler = QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end=4)


Directory already exists: model_output
